Project to test out usefulness of aggregated data when scaled given the high number
of outliers in the data

Import packages

In [ ]:
import pandas as pd
import numpy as numpy
import seaborn as sns
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')


Make data more readable by limiting decimal places in floats and show all columns

In [ ]:
pd.options.display.float_format = '{:20.2f}'.format
pd.set_option('display.max_columns', 999)

Load in and check the data

In [ ]:
df = pd.read_csv("C:\\Users\\alexd\\Python Projects\\k_means/aggregated_data.csv", encoding='ISO-8859-1')

agg_df = df.copy()
agg_df.head()

Plot the aggregated data using boxplots

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.boxplot(data=agg_df['MonetaryValue'], color='lightgreen')
plt.title('Monetary Value boxplot')
plt.xlabel('Monetary Value')
plt.ylabel('Count')

plt.subplot(1, 3, 2)
sns.boxplot(data=agg_df['Frequency'], color='skyblue')
plt.title('Frequency boxplot')
plt.xlabel('Frequency')
plt.ylabel('Count')

plt.subplot(1, 3, 3)
sns.boxplot(data=agg_df['Recency'], color='salmon')
plt.title('Recency')
plt.xlabel('Recency')
plt.ylabel('Count')

plt.ticklabel_format()
plt.show()

The data in the boxplots is essentially unreadable as the monetary value and frequency boxplots are squished down at the bottom of the plot. This is due to both of those features having a large number of high ranking processes that appear as extreme outliers. Those are high value customers so they will not be dropped. We will attempt scaling the data to see if that brings it together better.

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D



In [ ]:
# 3-D scatterplot the data to look at size of scale differences


fig = plt.figure(figsize= (8, 8))

ax = fig.add_subplot(projection='3d')

scatter = ax.scatter(agg_df['MonetaryValue'], agg_df['Frequency'], agg_df['Recency'])

ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3-D Scatterplot of Aggregated Data')

plt.show()




The 3-D scatterplot confirms that the data is squished to the front lower left.
The scaling for Frequency and Recency is comparable but Monetary Value is 1000x in scale

First scaling to use is StandardScaler. StandardScaler transforms data to a mean of 0 and std dev of 1.

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(agg_df[['MonetaryValue', 'Frequency', 'Recency']])
scaled_data

In [ ]:
#scaled_data is returned a numpy array so it needs to be converted back to pandas

scaled_data_df = pd.DataFrame(scaled_data, index=agg_df.index,
                              columns=('MonetaryValue', 'Frequency', 'Recency'))
scaled_data_df

New 3-D scatterplot of scaled_data_df using StandardScaler

In [ ]:
fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(projection="3d")

scatter = ax.scatter(scaled_data_df['MonetaryValue'], scaled_data_df['Frequency'], 
                     scaled_data_df['Recency'])

ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3-D Scatterplot of Std Scaled Customer Data')

plt.show()

The result of StandardScaler is not useful. The scaling is not standard and the data is still squished front, low left

The second trial will be to use MinMax scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(agg_df[['MonetaryValue', 'Frequency', 'Recency']])
scaled_data

In [ ]:
mm_scaled_data_df = pd.DataFrame(scaled_data, index=agg_df.index,
                                 columns=['MonetaryValue', 'Frequency', 'Recency'])
mm_scaled_data_df

In [ ]:
fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(projection="3d")

scatter = ax.scatter(mm_scaled_data_df['MonetaryValue'], mm_scaled_data_df['Frequency'], 
                     mm_scaled_data_df['Recency'])

ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3-D Scatterplot of MinMax Scaled Customer Data')

plt.show()

MinMax scaling is better but not sufficiently to use for analysis.
Third attempt will be to use RobustScaler - this notionally takes into account severe outliers which is the situation
here

In [ ]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
rs_scaled_data = scaler.fit_transform(agg_df[['MonetaryValue', 'Frequency', 'Recency']])
rs_scaled_data

In [ ]:
rs_scaled_data_df = pd.DataFrame(rs_scaled_data, index=agg_df.index,
                                 columns=['MonetaryValue', 'Frequency', 'Recency'])

rs_scaled_data_df

3-D scatterplot of Customer Data using data scaled by RobustScaler

In [ ]:
fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(projection="3d")

scatter = ax.scatter(rs_scaled_data_df['MonetaryValue'], rs_scaled_data_df['Frequency'], 
                     rs_scaled_data_df['Recency'])

ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3-D Scatterplot of RobustScaler Scaled Customer Data')

plt.show()

import numpy as np
From co-pilot transform the monetaryvalue column using a log transformation
df['MV'] = np.log1p(df['MV'])

np.log1p handles 0s more elegantly

copy agg_df to agg_log_df to try this out


In [ ]:
selected_columns = ['MonetaryValue', 'Frequency', 'Recency']

agg_log_df = agg_df[selected_columns].copy()

import numpy as np
agg_log_df['MonetaryValue'] = np.log1p(agg_log_df['MonetaryValue'])
agg_log_df['Frequency'] = np.log1p(agg_log_df['Frequency'])
agg_log_df['Recency'] = np.log1p(agg_log_df['Recency'])
agg_log_df[['MonetaryValue', 'Frequency', 'Recency']]

In [ ]:
fig = plt.figure(figsize= (8, 8))

ax = fig.add_subplot(projection='3d')

scatter = ax.scatter(agg_log_df['MonetaryValue'], agg_log_df['Frequency'], agg_log_df['Recency'])

ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3-D Scatterplot of Aggregated Data with Log Transformation')

plt.show()

KMneans clustering
First run k from 2-12 to determine number of clusters
Then run silhouette scores to find the correct number of clusters to use
Re-run KMeans on this cluster value

In [ ]:
max_k = 12

inertia = []
silhouette_scores = []
k_values = range(2, max_k + 1)  # clusters range 2-12

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, max_iter=1000)
    cluster_labels = kmeans.fit_predict(agg_log_df)
    sil_score = silhouette_score(agg_log_df, cluster_labels)
    silhouette_scores.append(sil_score)
    inertia.append(kmeans.inertia_)
    
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(k_values, inertia, marker='o')
plt.title('KMeans Inertia for Different Values of (k)')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.xticks(k_values)
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(k_values, silhouette_scores, marker='o', color='orange')
plt.title('Silhouette Scores for Different Values of (k)')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.xticks(k_values)
plt.grid(True)

plt.tight_layout()
plt.show()

The plots suggest that 4 is the optimal number of clusters and this is supported by the silhouette score
Work the method to calulate the cluster ids and add new column for them to agg_df

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42, max_iter=1000)
cluster_labels = kmeans.fit_predict(agg_log_df)
cluster_labels

In [ ]:
agg_log_df['Cluster'] = cluster_labels
agg_log_df

3-D plot of clusters by colour mapping to visualizee the results

In [ ]:
cluster_colors = {
    0: '#1f77b4', # Blue
    1: '#ff7f0e', # Orange
    2: '#2ca02c', # Green
    3: '#d62728', # Red
}

colors = agg_log_df['Cluster'].map(cluster_colors)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(projection='3d')

scatter = ax.scatter(agg_log_df['MonetaryValue'],
                     agg_log_df['Frequency'],
                     agg_log_df['Recency'],
                     c=colors,
                     marker = 'o')

ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3-D Scatterplot of Customer Data by Cluster')

plt.show()

Create violin plots of the clusters to determine placing for the 3 features.
The gray feature in the plot is the raw data

In [ ]:
plt.figure(figsize=(12, 18))

plt.subplot( 3, 1, 1)
sns.violinplot(x=agg_log_df['Cluster'], y=agg_log_df['MonetaryValue'], palette=cluster_colors,
               hue=agg_log_df['Cluster'])
sns.violinplot(agg_log_df['MonetaryValue'], color='gray', linewidth=1.0)
plt.title('Monetary Value by Cluster')
plt.ylabel('Monetary Value')

plt.subplot( 3, 1, 2)
sns.violinplot(x=agg_log_df['Cluster'], y=agg_log_df['Frequency'], palette=cluster_colors,
               hue=agg_log_df['Cluster'])
sns.violinplot(agg_log_df['Frequency'], color='gray', linewidth=1.0)
plt.title('Frequency by Cluster')
plt.ylabel('Frequency')

plt.subplot( 3, 1, 3)
sns.violinplot(x=agg_log_df['Cluster'], y=agg_log_df['Recency'], palette=cluster_colors,
               hue=agg_log_df['Cluster'])
sns.violinplot(agg_log_df['Recency'], color='gray', linewidth=1.0)
plt.title('Recency by Cluster')
plt.ylabel('Recency')

plt.tight_layout()
plt.show()

In [24]:
agg_log_df.to_csv("C:\\Users\\alexd\\Python Projects\\k_means/agg_data_log_scaled.csv", index=False, encoding='ISO-8859-1')